In [88]:
import sys
sys.path.append("..") ## resetting the path to the parent directory

In [89]:
from src.pipeline import *
import src.key_reader as kr

In [90]:
api_key = kr.key()

In [ ]:
import json
import requests

In [ ]:
with open("../data/matchlist_kr.json",'r') as file:
    kr_match_ids = json.load(file)

## Let's assume I already have a massive raw match details
## Obj. 1: build a function to grab only the necessary features.
### team stats in a given match:
'win': 'Win',
'firstBlood': True,
'firstTower': False,
'firstInhibitor': False,
'firstBaron': False,
'firstDragon': False,
'firstRiftHerald': False,
'towerKills': 9,
'inhibitorKills': 1,
'baronKills': 0,
'dragonKills': 2,
'vilemawKills': 0,
'riftHeraldKills': 0,
'dominionVictoryScore': 0

### player stats n a given match:
participantId': 1,
     'win': True,
     'item0': 3800,
     'item1': 3748,
     'item2': 3193,
     'item3': 3065,
     'item4': 3742,
     'item5': 3071,
     'item6': 3340,
     'kills': 17,
     'deaths': 5,
     'assists': 25,
     'largestKillingSpree': 8,
     'largestMultiKill': 2,
     'killingSprees': 3,
     'longestTimeSpentLiving': 1144,
     'doubleKills': 1,
     'tripleKills': 0,
     'quadraKills': 0,
     'pentaKills': 0,
     'unrealKills': 0,
     'totalDamageDealt': 335249,
     'magicDamageDealt': 0,
     'physicalDamageDealt': 331745,
     'trueDamageDealt': 3504,
     'largestCriticalStrike': 0,
     'totalDamageDealtToChampions': 42926,
     'magicDamageDealtToChampions': 0,
     'physicalDamageDealtToChampions': 42190,
     'trueDamageDealtToChampions': 736,
     'totalHeal': 5174,
     'totalUnitsHealed': 1,
     'damageSelfMitigated': 84529,
     'damageDealtToObjectives': 17163,
     'damageDealtToTurrets': 8614,
     'visionScore': 50,
     'timeCCingOthers': 16,
     'totalDamageTaken': 82758,
     'magicalDamageTaken': 35020,
     'physicalDamageTaken': 44500,
     'trueDamageTaken': 3237,
     'goldEarned': 26248,
     'goldSpent': 21700,
     'turretKills': 3,
     'inhibitorKills': 0,
     'totalMinionsKilled': 373,
     'neutralMinionsKilled': 12,
     'neutralMinionsKilledTeamJungle': 0,
     'neutralMinionsKilledEnemyJungle': 9,
     'totalTimeCrowdControlDealt': 388,
     'champLevel': 18,
     'visionWardsBoughtInGame': 0,
     'sightWardsBoughtInGame': 0,
     'wardsPlaced': 27,
     'wardsKilled': 5,
     'firstBloodKill': False,
     'firstBloodAssist': False,
     'firstTowerKill': False,
     'firstTowerAssist': False,
     'firstInhibitorKill': False,
     'firstInhibitorAssist': False

In [ ]:
with open("../data/matches1.json",encoding='latin1') as file:
    match_details = json.load(file) 

In [ ]:
match_details['matches'][0] ## <- each match detail in json file

In [ ]:
match_details['matches'][0]['participants'][0]['timeline'] ## <- each match detail in json file

In [ ]:
len(match_details['matches'])

In [ ]:
## Team stats
team_features = ['teamId','win','firstBlood','firstTower',
                 'firstInhibitor','firstBaron','firstDragon',
                 'firstRiftHerald','towerKills','inhibitorKills',
                 'baronKills','dragonKills','vilemawKills',
                 'riftHeraldKills','dominionVictoryScore']

In [ ]:
## team 0 and team 1
match_details['matches'][0]['teams'][0]

In [ ]:
import pandas as pd
match_df = pd.DataFrame(columns = team_features)

In [ ]:
#for i in range(len(match_details['matches'][0]['teams'])):
    

## individual stats from the provided seed datasets

In [ ]:
r = requests.get(url = 'https://s3-us-west-1.amazonaws.com/riot-developer-portal/seed-data/matches2.json')
raw_data = r.json()

In [ ]:
raw_data['matches'][0]['participants'][0]

In [ ]:
raw_data['matches'][0]['participants'][0]

In [ ]:
raw_data['matches'][0]['participants'][0]['highestAchievedSeasonTier']

In [ ]:
raw_data['matches'][0]['participants'][0]['timeline']['role']

In [ ]:
raw_data['matches'][0]['participants'][0]['timeline']['lane']

In [ ]:
def seed_csv():
    """
    This function extracts seed match data from the developer's s3 bucket
    and converts and saves as a csv file
    
    input:  empty list
    output: list of feature names for the csv file
    """
    import json
    import requests
    import pandas as pd
    from pathlib import Path
    
    ## Extracting from all 10 seed match data
    key_list = []
    match_stats_list = []

    for num in range(1,11):
        
        r = requests.get(url = f'https://s3-us-west-1.amazonaws.com/riot-developer-portal/seed-data/matches{num}.json')
        match_details = r.json()
        
        # check for and set up keys
        if len(key_list) < 1:
            for i in match_details['matches'][0]['participants'][0]['stats'].keys():
                key_list.append(i)
            key_list.append('highestAchievedSeasonTier')
            key_list.append('role')
            key_list.append('lane')
        # Remove unnecessary features

        print(f'match number: {num}')
        # populate match_stats_list 
        for match_i in range(len(match_details['matches'])) :
            
            for participant_i in range(len(match_details['matches'][match_i]['participants'])):
                match_stats_dict = {}
                
                for key in key_list:
                    try:
                        match_stats_dict[key] = match_details['matches'][match_i]['participants'][participant_i]['stats'][key]
                    except:
                        try: ## looking for highest achieved season tier
                            match_stats_dict[key] = match_details['matches'][match_i]['participants'][participant_i][key]
                        except:
                            try:  ## looking for role and lane
                                match_stats_dict[key] = match_details['matches'][match_i]['participants'][participant_i]['timeline'][key]
                            except: ## input null 
                                if not key in key_list:
                                    match_stats_dict[key] = None
                match_stats_list.append(match_stats_dict)
    
    # Converting the list to a pandas dataframe before saving it as a csv file
    my_file = Path("../data/seed_data.csv")
    
    if not my_file.is_file():
        match_stats_df = pd.DataFrame(match_stats_list)
        match_stats_df.to_csv('../data/seed_data.csv')    
        
    return key_list

In [ ]:
seed_df = seed_csv()

## Extracting match details from Korean match ids

In [ ]:
## api status check
import requests
r = requests.get(f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/faker?api_key={api_key}')
r.status_code

In [ ]:
def api_csv(api_key):
    import time
    import requests
    from pathlib import Path
    import pandas as pd
    """
    CALL LIMITS FOR RIOT GAMES API

    100 calls per second

    1000 calls per 10 seconds

    60,000 calls per 10 minutes (600 seconds)
    
    input: api key (list)
    output: pandas dataframe
    """
    # Requesting match IDs 
    r = requests.get(url = 'http://canisback.com/matchId/matchlist_kr.json')   
    match_ids = r.json()
    
    kr_keys = []

    for num, match_id in enumerate(match_ids):
    
        n = 0

        if n % 80 == 0:
            time.sleep(1)
            pass

        elif n % 900 == 0:
            time.sleep(10)
            pass

        elif n % 25000 == 0:
            time.sleep(600)
            pass

        else:
            pass
        
        match_stats_list = []

        # Calls API
        response = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/matches/{match_id}?api_key={api_key}')
        #print(response.status_code)


        # Counting API calls
        n+=1

        # Formating the api call 
        kr_match_details = response.json()
        
        # check for and set up feature names as keys
        if len(kr_keys) < 1:
            kr_keys = []
            for i in kr_match_details['participants'][0]['stats'].keys():
                kr_keys.append(i)
            kr_keys.append('highestAchievedSeasonTier')
            kr_keys.append('role')
            kr_keys.append('lane')
        #print(f'seed dataset number: {num}')

        ## Adding 
        for participant_i in range(len(kr_match_details['participants'])):
        #    print(f'participant number: {participant_i}')
            kr_match_stats_dict = {}

            for key in kr_keys:
                try:
                    kr_match_stats_dict[key] = kr_match_details['participants'][participant_i]['stats'][key]
                except:
                    try: ## looking for highest achieved season tier
                        kr_match_stats_dict[key] = kr_match_details['participants'][participant_i][key]
                    except:
                        try:  ## looking for roles and lanes
                            kr_match_stats_dict[key] = kr_match_details['participants'][participant_i]['timeline'][key]
                        except: ## input null 
                            if not key in key_list:
                                kr_match_stats_dict[key] = None

            match_stats_list.append(kr_match_stats_dict)            

        # Checking if a csv file exists before appending the data
        my_file = Path("../data/kr_data.csv")
        match_stats_df = pd.DataFrame(match_stats_list)

        # Create a csv file if it does not exist
        if not my_file.is_file():
            match_stats_df = pd.DataFrame(match_stats_list)
            match_stats_df.to_csv('../data/kr_data.csv')

        # Append data onto the existing csv file
        else:
            match_stats_df = pd.DataFrame(match_stats_list)
            match_stats_df.to_csv('../data/kr_data.csv',header=None,mode='a')
    return match_stats_df

In [ ]:
kr_df = api_csv(api_key=api_key)

In [ ]:
kr_data.columns

In [ ]:
response = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/matches/4366059781?api_key={api_key}')
a = response.json()

In [ ]:
a.keys()

In [ ]:
##### participant's user id for repeated plays
### dictionary['participantIdentities'][participant_i]['participantId']
### dictionary['participantIdentities'][0]['player']['accountId']
a['participantIdentities'][0]

In [ ]:
a['teams']

In [62]:
response_t = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/4366059781?api_key={api_key}')

In [ ]:
b = response_t.json()

In [ ]:
#keys = ['participantId','totalGold','level','xp','killerId','victimId']
keys = ['totalGold','level','xp','totalKills','totalDeaths']

participant_ids = [1,2,3,4,5,6,7,8,9,10]

team1_halftime_stats = {'match_id': '4366059781','totalGold':0,'level':0,
                        'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}

team2_halftime_stats = {'match_id': '4366059781','totalGold':0,'level':0,
                        'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}

# Taking first half of the match stats to predict the match outcome.

for frame_i in range(len(b['frames'])//2):
    
    ## Grabbing ingame gold, levels, xp    
    for p_i in range(1,len(participant_ids)+1):
        for ind_key in keys[:-2]:
            if b['frames'][frame_i]['participantFrames'][str(p_i)]['participantId'] <= 5:
                team1_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]
            else:
                team2_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]
    
    ## Grabbing ingame kills and deaths
    for event_i in range(len(b['frames'][frame_i]['events'])):
        for ind_key2 in keys[-2:]:
            
            if b['frames'][frame_i]['events'][event_i]['type'] == 'CHAMPION_KILL':
                if b['frames'][frame_i]['events'][event_i]['killerId'] <= 5:
                    team1_halftime_stats['totalKills'] += 1
                    team2_halftime_stats['totalDeaths'] += 1
                else:
                    team2_halftime_stats['totalKills'] += 1
                    team1_halftime_stats['totalDeaths'] += 1       
            if b['frames'][frame_i]['events'][event_i]['type'] =='SKILL_LEVEL_UP':
                if b['frames'][frame_i]['events'][event_i]['participantId'] <= 5:
                    team1_halftime_stats['level_ups'] += 1
                else:
                    team2_halftime_stats['level_ups'] += 1

In [ ]:
team2_halftime_stats

In [ ]:
['win': 'Fail','firstBlood', 'firstTower','firstInhibitor': False,'firstBaron': False,'firstDragon','firstRiftHerald']

In [ ]:
a['teams'][0].keys()

In [ ]:
#additional_keys = a['teams'].keys()
#additional_keys = additional_keys[1:8]

for team_i in range(len(a['teams'])):
    if a['teams'][team_i]['teamId'] == 100:
        team1_halftime_stats['win'] = a['teams'][team_i]['win']
        team1_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
        team1_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
        team1_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
        team1_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
        team1_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
        team1_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']
        
    else:
        team2_halftime_stats['win'] = a['teams'][team_i]['win']
        team2_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
        team2_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
        team2_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
        team2_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
        team2_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
        team2_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']

In [ ]:
team1_halftime_stats

In [ ]:
team2_halftime_stats

## Combination of first half match data from 2 api calls

In [50]:
import json
with open('../data/matchlist_kr.json','r') as file:
    data = json.load(file)

In [63]:
def halftime_report(maapi_key=api_key):
    import time
    import requests
    from pathlib import Path
    import pandas as pd
    
    """
    CALL LIMITS FOR RIOT GAMES API

    20 calls per second
    
    100 calls per 120 seconds
    
    60,000 calls per 10 minutes (600 seconds)
    
    input: api key (list)
    output: pandas dataframe
    """
    
    # Requesting match IDs 
    
    #r = requests.get(url = 'http://canisback.com/matchId/matchlist_kr.json')   
    #match_ids = r.json()
    
    n = 519
    
    api_error_n = 0
    
    match_ids = match_ids[519:]
    
    for match_id in match_ids:
    
        if n % 2 == 0:
            time.sleep(4)
            pass
        
        if n % 15 == 0:
            time.sleep(20)
            pass
        
        elif n% 35 == 0:
            time.sleep(60)
            pass
        
        elif n % 70 == 0:
            time.sleep(120)
            pass
        
        elif n % 110 == 0:
            time.sleep(300)
            pass
    
        elif n % 130 == 0:
            time.sleep(500)
            pass
        
        elif n % 235 == 0:
            time.sleep(600)
            pass
        
        response_match = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/matches/{match_id}?api_key={api_key}')
        
        time.sleep(1)
        
        response_t = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{match_id}?api_key={api_key}')

        n += 2

        a = response_match.json()
        b = response_t.json()

        keys = ['totalGold','level','xp','totalKills','totalDeaths']

        team1_halftime_stats = {'match_id': {match_id},'totalGold':0,'level':0,
                                'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}

        team2_halftime_stats = {'match_id': {match_id},'totalGold':0,'level':0,
                                'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}
        halftime_stats = []

        # Taking first half of the match stats to predict the match outcome.
        try:
            if response_match.status_code == 200 and response_t.status_code == 200:
                for frame_i in range(len(b['frames'])//2):

                    ## Grabbing ingame gold, levels, xp    
                    for p_i in range(1,len(participant_ids)+1):
                        for ind_key in keys[:-2]:
                            if b['frames'][frame_i]['participantFrames'][str(p_i)]['participantId'] <= 5:
                                team1_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]
                            else:
                                team2_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]

                    ## Grabbing ingame kills and deaths
                    for event_i in range(len(b['frames'][frame_i]['events'])):
                        for ind_key2 in keys[-2:]:

                            if b['frames'][frame_i]['events'][event_i]['type'] == 'CHAMPION_KILL':
                                if b['frames'][frame_i]['events'][event_i]['killerId'] <= 5:
                                    team1_halftime_stats['totalKills'] += 1
                                    team2_halftime_stats['totalDeaths'] += 1
                                else:
                                    team2_halftime_stats['totalKills'] += 1
                                    team1_halftime_stats['totalDeaths'] += 1       
                            if b['frames'][frame_i]['events'][event_i]['type'] =='SKILL_LEVEL_UP':
                                if b['frames'][frame_i]['events'][event_i]['participantId'] <= 5:
                                    team1_halftime_stats['level_ups'] += 1
                                else:
                                    team2_halftime_stats['level_ups'] += 1

                ## Adding additional ingame data
                for team_i in range(len(a['teams'])):
                    if a['teams'][team_i]['teamId'] == 100:
                        team1_halftime_stats['teamId'] = 1
                        team1_halftime_stats['win'] = a['teams'][team_i]['win']
                        team1_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
                        team1_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
                        team1_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
                        team1_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
                        team1_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
                        team1_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']

                    else:
                        team2_halftime_stats['teamId'] = 2
                        team2_halftime_stats['win'] = a['teams'][team_i]['win']
                        team2_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
                        team2_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
                        team2_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
                        team2_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
                        team2_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
                        team2_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']
                
                print(team2_halftime_stats)
                halftime_stats.append(team1_halftime_stats)
                halftime_stats.append(team2_halftime_stats)

                # Checking if a csv file exists before appending the data
                my_file = Path("../data/ht_match.csv")

                # Create a csv file if it does not exist
                if not my_file.is_file():
                    match_stats_df = pd.DataFrame(halftime_stats)
                    match_stats_df.to_csv('../data/ht_match.csv')
                    time.sleep(10)

                # Append data onto the existing csv file
                else:
                    match_stats_df = pd.DataFrame(halftime_stats)
                    match_stats_df.to_csv('../data/ht_match.csv',header=None,mode='a')
                    time.sleep(10)
        except:
            print(api_error_n)
            api_error_n += 1

            if (response_match.status_code == 429 or response_t.status_code ==429) and api_error_n < 3:
                time.sleep(300)
                pass

            elif (response_match.status_code == 429 or response_t.status_code ==429) and temp_n >= 3 and api_error_n < 7:
                time.sleep(500)
                pass

            elif api_error_n >= 7:
                print('breaking due to excessive number of pulls')
                break

            else:
                print(response_match.status_code)
                break


In [57]:
match_ids = '4366109430'

In [91]:
def halftime_team_report(api_key=api_key):
    import time
    import requests
    from pathlib import Path
    import pandas as pd
    
    """
    CALL LIMITS FOR RIOT GAMES API

    20 calls per second
    
    100 calls per 120 seconds
        
    input: api key (list)
    output: pandas dataframe
    """
    
    # Requesting match IDs 
    
    with open('../data/matchlist_kr.json','r') as file:
        match_ids = json.load(file)
    
    n = 9494
    
    api_error_n = 0
    
    match_ids = match_ids[9494:]
    
    for match_id in match_ids:
    
#        if n % 8 == 0:
#            time.sleep(1)
#            pass
        
        if n % 45 == 0:
            time.sleep(125)
            pass
          
        response_match = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/matches/{match_id}?api_key={api_key}')
        print(n)
        n += 1

        a = response_match.json()

        team1_halftime_stats = {'match_id': {match_id}}

        team2_halftime_stats = {'match_id': {match_id}}

        # Taking first half of the match stats to predict the match outcome.
        try:
            ## Adding additional ingame data
            halftime_stats = []

            for team_i in range(len(a['teams'])):
                if a['teams'][team_i]['teamId'] == 100:
                    team1_halftime_stats['teamId'] = 1
                    team1_halftime_stats['win'] = a['teams'][team_i]['win']
                    team1_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
                    team1_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
                    team1_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
                    team1_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
                    team1_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
                    team1_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']

                else:
                    team2_halftime_stats['teamId'] = 2
                    team2_halftime_stats['win'] = a['teams'][team_i]['win']
                    team2_halftime_stats['firstBlood'] = a['teams'][team_i]['firstBlood']
                    team2_halftime_stats['firstTower'] = a['teams'][team_i]['firstTower']
                    team2_halftime_stats['firstInhibitor'] = a['teams'][team_i]['firstInhibitor']
                    team2_halftime_stats['firstBaron'] = a['teams'][team_i]['firstBaron']
                    team2_halftime_stats['firstDragon'] = a['teams'][team_i]['firstDragon']
                    team2_halftime_stats['firstRiftHerald'] = a['teams'][team_i]['firstRiftHerald']
                        
            halftime_stats.append(team1_halftime_stats)
            halftime_stats.append(team2_halftime_stats)
            
            # Checking if a csv file exists before appending the data
            my_file = Path("../data/ht_match.csv")

            # Create a csv file if it does not exist
            if not my_file.is_file():
                match_stats_df = pd.DataFrame(halftime_stats)
                match_stats_df.to_csv('../data/ht_match.csv')
                time.sleep(0.125)
                halftime_stats = []
            # Append data onto the existing csv file
            else:
                match_stats_df = pd.DataFrame(halftime_stats)
                match_stats_df.to_csv('../data/ht_match.csv',header=None,mode='a')
                time.sleep(0.125)
                halftime_stats = []
        

        except:
            print(response_match.status_code)
            print(f"n'th match: {n}")
            print(api_error_n)
            api_error_n += 1

            if (response_match.status_code == 429) and api_error_n < 3:
                time.sleep(30)
                pass

            elif (response_match.status_code == 429) and api_error_n >= 3 and api_error_n < 7:
                time.sleep(30)
                pass

            elif api_error_n >= 7:
                print('breaking due to excessive number of pulls')
                break
            
            elif response_match.status_code == 504 or response_t.status_code == 503:
                api_error_n -=1
                time.sleep(30)
                pass
            
            else:
                print(response_match.status_code)
                time.sleep(30)
                pass

In [92]:
halftime_team_report()

9494
9495
9496
9497
9498
9499
9500
9501
9502
9503
9504
9505
9506
9507
9508
9509
9510
9511
9512
9513
9514
9515
9516
9517
9518
9519
9520
9521
9522
9523
9524
9525
9526
9527
9528
9529
9530
9531
9532
9533
9534
9535
9536
9537
9538
9539
9540
9541
9542
9543
9544
9545
9546
9547
9548
9549
9550
9551
9552
9553
9554
9555
9556
9557
9558
9559
9560
9561
9562
9563
9564
9565
9566
9567
9568
9569
9570
9571
9572
9573
9574
9575
9576
9577
9578
9579
9580
9581
9582
9583
9584
9585
9586
9587
9588
9589
9590
9591
9592
9593
9594
9595
9596
9597
9598
9599
9600
9601
9602
9603
9604
9605
9606
9607
9608
9609
9610
9611
9612
9613
9614
9615
9616
9617
9618
9619
9620
9621
9622
9623
9624
9625
9626
9627
9628
9629
9630
9631
9632
9633
9634
9635
9636
9637
9638
9639
9640
9641
9642
9643
9644
9645
9646
9647
9648
9649
9650
9651
9652
9653
9654
9655
9656
9657
9658
9659
9660
9661
9662
9663
9664
9665
9666
9667
9668
9669
9670
9671
9672
9673
9674
9675
9676
9677
9678
9679
9680
9681
9682
9683
9684
9685
9686
9687
9688
9689
9690
9691
9692
9693
